# The nature of recursion

In this tutorial we get acquainted with recursion. It may be difficult to understand first, may be harder to get used to it, but once it is deeply conceived, it becomes not only a powerful tool, but also a new way of thinking.

All examples and explantions will be given in the context of Python programming language. The language is irrelevant, key principles are vital.

## Table of contents

- [Recognition](#recognition)

- [Definition](#definition)

- [Recursion in Computer Science](#recursion-in-computer-science)

- [Stack](#stack)

- [Examples of Recursion](#examples-of-recursion)

    - [Factorial](#factorial)
    - [The sum of numbers](#the-sum-of-numbers)
    - [Reversing a string](#reversing-a-string)
    - [Triangular numbers](#triangular-numbers)

- [Homework](#homework)

- [References](#references)

- [Solutions](#solutions)

Attention! This note import the [`pytest`](https://docs.pytest.org/en/7.4.x/) testing package. It can be installed, for example, via `pip3 install pytest` command.

In [1]:
from contextlib import nullcontext as does_not_raise
from typing import Any

import ipytest
import pytest

# https://github.com/chmp/ipytest
ipytest.autoconfig()

In [2]:
def validate_integer(number: Any) -> None:
    """Validates if an object is of int() type."""

    if not isinstance(number, int):
        raise TypeError(f"{number} is not int")


def validate_non_negative_integer(number: int) -> None:
    """Validates a number to be a non-negative integer."""

    validate_integer(number)
    if number < 0:
        raise ValueError(f"{number} < 0")


def validate_positive_integer(number: Any) -> None:
    """Validates a number to be a positive integer."""

    validate_integer(number)
    if number < 1:
        raise ValueError(f"{number} < 1")

### Recognition

Look at the following picture:

![Recursion that recurs](./recursion_that_recurs.jpg)

There is a certain self-repeating element, in this case, a kind of bracket with the inscription “Recursion it recurs" in the middle of it and so on, decreasing in size and moving away to the center of perspective, perhaps indefinitely.

Consider another example, the [Sierpiński triangle](https://en.wikipedia.org/wiki/Sierpi%C5%84ski_triangle):

![Sierpiński triangle](./Sierpinski_triangle_evolution.png)

The evolution of a Sierpiński triangle:

1. there is an equilateral triangle;

2. the midpoints of opposite sides are connected in such a way as to form an inner triangle which is "upside-down" with respect to the original triangle;

3. the same action is repeated for every smaller versions of the original triangle and so forth...

Again, self-repetition is striking: we see repeating triangles formed via the same procedure.

Finally, a jolly good example of a recursive Russian doll:

![Recursive Russian Doll](./recursive_russian_doll.png)

Recursion is also found in [reccurence relations](https://en.wikipedia.org/wiki/Recurrence_relation). A reccurence relation is an equation that defines the nth term of a sequence of numbers as a combination of its previous terms. Some of canonical examples are:

1. the [factorial](https://en.wikipedia.org/wiki/Factorial) of a non-negative integer **n**: `n! = n * (n - 1)!`, where 0! = 1! = 1;

2. the [Fibonacci sequence](https://en.wikipedia.org/wiki/Fibonacci_sequence): `F(n) = F(n - 1) + F(n - 2)`, where F(0) = 0 and F(1) = 1;

Examples on the above examples:

1. the factorial of 4:

    - 4! = 4 * 3!
    - 3! = 3 * 2!
    - 2! = 2 * 1!
    - 1! = 1 and we stop
    - therefore, 4! = 4 * 3 * 2 * 1 = 24.

2. the fourth (or maybe the fifth because of zero-index start) Fibonacci number

    - F(4) = F(4 - 1) + F(4 - 2) = F(3) + F(2)
    - F(3) = F(3 - 1) + F(3 - 2) = F(2) + F(1)
    - F(2) = F(2 - 1) + F(2 - 2) = F(1) + F(0)
    - we know, that F(0) = 0 and F(1) = 1 and we stop
    - so, F(2) = 1 + 0 = 1
    - and F(3) = F(2) + F(1) = 1 + 1 = 2
    - therefore, F(4) = 2 + 1 = 3

The definition of the factorial of a non-negative integer relies on a self-definition. To compute an `n!` we need to multiply `n` by `(n - 1)!`, but the latter term is the factorial by nature. The same case is for a number from the Fibonacci sequence when n > 1.

One major detail for both examples above: the computation of `n!` or `F(n)` requires a finite procedure, in other words, the case where to stop unwinding the [recursive definition](https://en.wikipedia.org/wiki/Recursive_definition). Otherwise, no stop, no final result. However, **the recursion is not constrained to be finite**. Moreover, [fractals](https://en.wikipedia.org/wiki/Fractal) are potentially infinite structures like this one:

![A fractal](./fractal_example.jpeg)

### Definition

What all the previous examples have in common? Repetition is striking. But if it is just repetition, why calling it recursion? The above examples were about recursion and repeating patterns are (maybe) easy to spot. So we can say something *"if recursion, then repetition"*. Certainly it’s not entirely reliable to judge recursion based on only three examples, but for now there are conclusions that everyone can draw their own.

For these concepts to be synonymous, which in our case means finding out their equivalence, it is necessary to evaluate the truth of the following proposition: *"if repetition, then recursion"*. But why this? Because a statement like "if A, then B" means that A is a subset of B, which means whenever an element is from A, it is automatically from B. When both "if A, then B" and "if B, then A" statements are true, it means that A = B. Why again? Mathematics, set theory branch, or something like that can be drawn from books on logic.

Consider the following counterexample:

1. initially, we have a letter, let's pick up "R"
2. then just repeating it -> R R
3. again -> R R R
4. and so on...

Can we say for sure that this was the case of recursion? Repetition, yes, recursion, not everything is so clear here. The trick part here is that we can obtain such a sequence with a recursive algorithm, or you may be so used to recursion that even here you can see it if you want.

Instead of defining the recursion now, let's take another strategy:

1. for now we can live without a formal definition

2. we play with recursion and gradually reveal its features - starting from recognition, remaining on intuitive understanding, expanding the number of ways of explaining what the recursion is and making our way through less fuzzy definitions in favour of more clear ones

I like this approach for its successive nature which may lead to a certain success (I hope so). This can help us not to reinvent the wheel and save enough time and energy without being strayed away. If you are still impatient or discontent, this [Recursion](https://en.wikipedia.org/wiki/Recursion) article is a not bad start.

What might look plausible or even true the properties of recursion:

- recursion may be finite or infinite, so this is not a key feature, but there is a sort of iteration (they are like heads and tails, but later)

- there is a repeating element that is similar to other ones - yes and they are not only to be smaller in size, the idea is they are different in non-essential characteristics so we can claim them being practically the same, or, in other words, similar;

- later you may see that recursive algorithms tend to solve a problem by reducing it to similar smaller subproblems, but shrinking is only one way to go - we can choose the opposite direction towards recurring with expansion unless a certain condition is reached (see triangular numbers [example](#))

In short, if we cannot say that recursion is only repetition and the given properties does not allow us to define the recursion concept, let's guess that a material feature of recursion is **self-reference** or defining itself with referencing itself.

### Recursion in Computer Science

In computer science, a class of objects or methods exhibits recursive behavior when it can be defined by two properties:

- a *base case* (or cases) - a terminating scenario that does not use recursion to produce and answer

- a *recursive step* - a set of rules that redices all successive cases towards the base case

In case of factorials:

- base cases are 0! = 1 or 1! = 1 -> no need to recur, the answer can be produced with returning these values

- the recursive step is to take n and multiply it by the factorial of (n - 1) and by this we are getting closer to the base case.

In computer science the recursion is something that is meant to be finite, **in general, the recursion does not have to be finite**. Because the resource of a computer are limited, infinite kinds of recursion are impractical. Moreover, the following simple example without any base case will cause an error.

In [3]:
def recur_unstoppably():
    """This function has no base case."""
    # self-reference = calling the defined function itself
    recur_unstoppably()

# invoking the function and...crash
recur_unstoppably()

RecursionError: maximum recursion depth exceeded

Down as expected. This is the case of [stack overflow](https://stackoverflow.com/questions/26158/how-does-a-stack-overflow-occur-and-how-do-you-prevent-it). In computer science, a [stack](https://en.wikipedia.org/wiki/Stack_(abstract_data_type)) is an [abstract data type](https://en.wikipedia.org/wiki/Abstract_data_type) (ADT). When an ADT incarnates into a concreate implementation it is called a data structure (DS).

Looks perplexing, here comes a simple example on [natural numbers](https://en.wikipedia.org/wiki/Natural_number):

- nature -> numbers that are naturally arise when counting things - they are integral and successive

- possible values -> 1, 2, 3, ... (but there are natural numbers with zero - you may refer to this [page](https://en.wikipedia.org/wiki/Natural_number#Notation)) - all positive or non-negative integer numbers

- possible operations: addition and multiplication on any two natural number produce a natural number. Subtraction is not always possible: if a - b and a < b, then we are out of range. The division is from the same row when getting fractions and so on.

So, when grouping data values specified by a set of possible values and allowed operation and/or a representation of these values, we form a data type, in our example a data type of natural numbers. The more abstract (detail-independent) the definition is, the more this data type is abstract. Since the mathematics is a very abstract science, our natural numbers data type can be referred to as an abstract data type because they fit into a mathematical model of natural numbers.

As a homework, you can implement the data type of natural numbers and by this have a data structure for this type.

### Stack

As mentioned in the previous section, a [stack](https://en.wikipedia.org/wiki/Stack_(abstract_data_type)) is an abstract data type. It serves as a collection of elements described as **last in, first out** (LIFO) or **first in, last out** (FILO) with the following operations:

- Necessary operations:

    - `push`, which adds an alement to the collection

    - `pop`, which removes the most recently added element

- Extra operations:

    - `peek`, which returns the last added value without modifying the stack

By this we have defined a stack as an abstract data type: the essence (LIFO/FILO collection) and possible operations (push/pop as primary and some extra for usability). We don't need to enumerate possible values because they are just items, meaning objects. In Python there is a built-in [list](https://docs.python.org/3/tutorial/datastructures.html#more-on-lists) data structure that can be [used as a stack](https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-stacks).

In [4]:
class Stack:
    """A class is a user-defined (data) type.

    A class defines the set of operations (methods)
    and variables (states) that will be inherent to
    the instances (objects) of this class.
    """

    # this is a class variable
    # class variables are shared among all instances of a class
    # here only for a demonstrative purpose
    answer = 42

    def __init__(self):
        # _stack is a variable of a concrete instance (self)
        # a prefix underscore signifies that _stack variable is private
        # a private variable is not meant to be accessed from the outer code
        # in Python, there is no such access level ensurance
        self._stack = []  # a list under the hood

    def __len__(self) -> int:
        """This is an implementational detail.

        This double-underscore (dunder) method
        allows to pass the instance of this class
        as a parametre for len() builtin function.

        This is an extension for Stack data structure.
        """
        return len(self._stack)
    
    def __str__(self) -> str:
        """Returns the string representation of a Stack instance.

        This is also for convenience as __len__(self) method.
        """
        cls_name = self.__class__.__name__
        return f"{cls_name}({self._stack})"

    def push(self, item: Any):
        self._stack.append(item)

    def pop(self) -> Any:
        return self._stack.pop()
    
    @property
    def peek(self) -> Any:
        """A propery in Python is a method.

        But we can access it as it were a mere instance variable.
        """
        return self._stack[-1]

Let's play and test it. If you have read about Python lists, you know that a stack is a poorer data structure...or more restrict if you like it better.

In [5]:
stk1 = Stack()
stk2 = Stack()

print("Demonstrating class variables")
print(f"stk1 ({stk1.answer}) and stk2 ({stk2.answer})\n")

stk1.push("hello")
stk2.push("world")

print("Demonstrating instance variables")
# code smell: accessing a "private" variable
print(f"stk1 ({stk1._stack}) and stk2 ({stk2._stack})\n")

stk1.push(stk2.pop())

print(f"{stk2}, len = {len(stk2)}")  # __str__ and __len__ methods
print(f"{stk1}: peek = {stk1.peek}, len = {len(stk1)}")

Demonstrating class variables
stk1 (42) and stk2 (42)

Demonstrating instance variables
stk1 (['hello']) and stk2 (['world'])

Stack([]), len = 0
Stack(['hello', 'world']): peek = world, len = 2


Why still care about a stack? Because when a function/routine is called/invoked, all necessary information about it is pushed into a [call stack](https://en.wikipedia.org/wiki/Call_stack). If a rotine calls another subroutine, a new frame is pushed into the call stack and so on if your code keeps doing so. When a function finishes and returns the control back to the higher level, the corresponding frame is popped from the call stack.

Usually, the OS and the programming language manage the stack, so ot out of your hands. But this is all what is managed for you, with recursion and improper design blowing the call stack is feasible enough. In Python, you can change the default recursion limit, but think at least twice before you change it.

### Examples of Recursion

The examples from this section do have something in common apart their recursive implementations. It does not have to do with "better" recursive implementations, but relates to (postmature) optimisation. Can you guess what it is?

#### Factorial

The factorial of a non-negative integer number - a classic problem when learning recursion.

In [6]:
def factorial_recursive(number: int) -> int:
    """Returns the factorial of a non-negative integer."""

    def _rec_fact(nbr: int) -> int:
        """The actual recursive factorial algorithm.

        This is a hack.
        Python allows define a functions in the body of another one.
        This might be good for the reasons as follows:
            - validate/preprocess data before moving to the algorithm
            - do it once and do not repeat in every call
        """

        # base case
        if nbr in (0, 1):
            return 1
        # recursive step
        return nbr * _rec_fact(nbr - 1)

    validate_non_negative_integer(number)
    return _rec_fact(number)

The validation function is crucial for preventing negative numbers being passed as a parametre. If a negative number sinks into the current implementation without validation, the base case is never reached resulting in RecursionError which means a stack overflow situation.

It is also possible to cause the RecursionError if passing the number that will definitely exceed the current recursion level. However, the computer can freeze earlier than hitting the recursion limit, the resources like CPU memory and so on are depletable.

Testing the recursive version of the factorial.

In [7]:
%%ipytest

# https://github.com/chmp/ipytest
@pytest.mark.parametrize(
    ("number", "answer", "expectation"),
    [
        (-1, None, pytest.raises(ValueError)),
        (0, 1, does_not_raise()),
        (1, 1, does_not_raise()),
        (4, 24, does_not_raise()),
        (5.0, 120, pytest.raises(TypeError)),
        # increase if you dare
        (5000, None, pytest.raises(RecursionError))
    ]
)
def test_factorial(number, answer, expectation):
    with expectation:
        result = factorial_recursive(number)
        answer == result

......                                                                                       [100%]
6 passed in 0.02s


You can add `print()` statements in the `recursive_factorial` function to "textualise" the function calls - it may be illustrative enough for "emulating" call stack responses. for example:

#### The sum of numbers

Or, the sum of a sequence of numbers, not only integer ones.

Yes, we can think about summing the numbers with recursion: `S(n) = n + S(n - 1)`. Reminds the factorial, but deals with addition operator - this is our recursive step, a uniform operation forging a repeating pattern.

But when to stop? The convention about an empty sequence is that its sum is zero. So, we can stop either when a sequence holds a single element or when it is empty with no elements at all. The second variant is preferrable because it covers empty sequences as well.

In [8]:
from typing import Sequence


def sum_recursively(seq: Sequence) -> int:
    """Returns the sum of the items from the sequence."""

    def _sum_rec(_seq: list, last_index: int) -> int:
        """The real recursive implementation.

        The implementation required a new parametre.
        The signature of the `sum_recursively` function is good
        and must not be changed - this is fixed and full stop.
        At last, the conversion to the list should be done at most once.
        """

        if last_index:
            # recursive step
            new_last_index = last_index - 1
            last_element = _seq[last_index - 1]
            return last_element + _sum_rec(_seq, new_last_index)
        # base case: for an empty sequence the sum is 0 conventionally
        return 0

    # for convenience and protection from:
    #    - non iterable objects
    #    - generators (similar but different nature/behaviour)
    lst = list(seq)
    return _sum_rec(lst, len(lst))

In [9]:
%%ipytest

# https://github.com/chmp/ipytest
@pytest.mark.parametrize(
    ("sequence", "answer", "expectation"),
    [
        ([], 0, does_not_raise()),
        (1, 1, pytest.raises(TypeError)),
        ([1], 1, does_not_raise()),
        ((1, 2, 3), 6, does_not_raise()),
    ]
)
def test_sum_rec(sequence, answer, expectation):
    with expectation:
        result = sum_recursively(sequence)
        answer == result

....                                                                                         [100%]
4 passed in 0.01s


#### Reversing a string

A string is a sequence of characters. A character is a symbol from a base set called alphabet. A string containing no character is an empty string.

The task is to reverse a string. For instance, if we have "abc" string, its reverse will be "cba" string. How to think about this process recursively?

- if S = "" or "1", then R = "" or "1" and R = S, so we can just return S -> base cases, but an empty string seems to be more base in this case;

- if S = "12", then R = "21" and so on - the length of the string is greater than two (or one if keeping in mind the previous point) -> recursive part

It remains to design a recursive step and it is composite:

1. there is a string of length n -> S(n);

2. if the string is not empty, the last character (Lc) is torn off the string - we have a new string S(n - 1) and Lc;

3. reordering: instead of `S(n - 1) + Lc` we have `Lc + S(n - 1)`, where plus (+) sign means string concatenation operation

Done)

In [10]:
def reverse_string_recursive(string: str) -> str:
    """Returns the reversed string.
    
    The function is made to be pure.
    It comes from the functional programming paradigm.
    A pure function is meant to have no side effects.
    In this case our function does not affect the original string.
    """

    def _reverse_string(s1: str, idx: int) -> str:
        """The actual recursive routine."""

        if not idx:
            return ""
        return s1[idx - 1] + _reverse_string(s1, idx - 1)

    if not isinstance(string, str):
        raise TypeError(f"{string} is not of type 'str'")
    return _reverse_string(string, len(string))

In [11]:
%%ipytest

# https://github.com/chmp/ipytest
@pytest.mark.parametrize(
    ("string", "answer", "expectation"),
    [
        ("", "", does_not_raise()),
        ("a", "a", does_not_raise()),
        ("ab", "ba", does_not_raise()),
        (('a', 2), (2, 'a'), pytest.raises(TypeError)),
    ]
)
def test_reverse_string_recursively(
    string, answer, expectation
):
    with expectation:
        result = reverse_string_recursive(string)
        answer == result

....                                                                                         [100%]
4 passed in 0.01s


#### Triangular numbers

This example will be designed in a specific way to demonstrate that a recursive algorithm does not necessarily mean to always reduce a problem to a smaller subproblems. The fact is that the recursion can work towards expansion and look at a problem at a different angle. And about angles and a bit of geometry is this example on triangular numbers.

A triangular number is such a number that, if taking the same amount of uniform objects as the value of the given triangular number, then we can form of these objects a triangle. Visualising triangular and other "figurative" numbers with the following picture:

!["Figurative numbers"](./figurative_numbers.jpg)

The idea is to think these numbers recursively, but in both directions:

1. shrinking -> the nth triangular number is defined in terms of the previous -> `T(n) = T(n - 1) + n` and the base case is `T(1) = 1`;

2. expanding -> `T(k + 1) = T(k) + (k + 1)` and the recursive step keeps going until `(k + 1) != n`, or "gets equal to the nth term".

What? This is weird, cannot be though seriously, to be discarded...but think about it, look at the above picture again, try to play with other geometric numbers and behold that you can move not only from the right to the left, from a problem to a subproblem, but also in the opposite direction. Certainly, practicality beats purity and you pick up what is less exhaustive to understand and maintain.

In [12]:
def get_nth_triangular_number(nth: int) -> int:
    """Returns the nth triangular number.

    The indexation starts from zero, so T(0) = 1, T(1) = 3 and so forth.
    The recursive algorithm is a "expanding recursion" (unofficial, yikes).

    Yep, bad examples can ruin good ideas.
    But this example may seem bad so far...
    """

    def _triangulate(limit: int, current: int) -> int:
        if not limit:
            return current
        # the answer variable is accumulator
        return current + _triangulate(limit - 1, current + 1)

    validate_non_negative_integer(nth)
    return _triangulate(nth, 1)

In [13]:
%%ipytest

# https://github.com/chmp/ipytest
@pytest.mark.parametrize(
    ("number", "answer", "expectation"),
    [
        (-21, None, pytest.raises(ValueError)),
        (4.2, None, pytest.raises(TypeError)),
        (0, 1, does_not_raise()),
        (1, 3, does_not_raise()),
        (2, 6, does_not_raise()),
        (3, 10, does_not_raise()),
    ]
)
def test_get_nth_triangular_number(
    number, answer, expectation
):
    with expectation:
        result = get_nth_triangular_number(number)
        answer == result

......                                                                                       [100%]
6 passed in 0.01s


### Homework

This is an extra section for those, who like doing some home work.

0. consider learning [pytest](https://docs.pytest.org/en/latest/) framework for writing unit tests in an easy-coding fashion. For the following home tasks unit tests are mandatory.

1. Write a class `NaturalNumber` which represents a natural number ADT. You'll need to implement the following methods for supporting:

    - addition: `__add__`, `__iadd__` and `__radd__`;

    - multiplication: `__mul__` and so forth (see addition);

    - subtraction: `__sub__` et cetera. The subtraction should work for b - a only if b > a;

    - division: you'll have to find the corresponding dunders for this operation + the division is possible if the result belongs to the set of natural numbers.

2. Write a recursive implementation for computing the nth term of the Fibonacci sequence. The base cases are `F(0) = 0` and `F(1) = 1`.

3. Write iterative versions for examples from the [Examples of Recursion](#examples-of-recursion) section.

### References

- [WikiPedia -> Recursion](https://en.wikipedia.org/wiki/Recursion)

- [YouTube -> Recursion for Beginners: A Beginner's Guide to Recursion - Al Sweigart](https://www.youtube.com/watch?v=AfBqVVKg4GE)

- [YouTube -> Recursion - V. Anton Spraul (Think Like a Programmer)](https://www.youtube.com/watch?v=oKndim5-G94)

- [YouTube -> FreeCodeCamp - Recursion in Programming](https://www.youtube.com/watch?v=IJDJ0kBx2LM)

- [enjoyalgorithms.com -> Fundamentals of Recursion in Programming](https://www.enjoyalgorithms.com/blog/recursion-explained-how-recursion-works-in-programming)

- [educative.io -> Recursion: A Quick Guide for Software Engineers](https://www.educative.io/blog/recursion)

- [WikiPedia -> Stack](https://en.wikipedia.org/wiki/Stack_(abstract_data_type))

- [Baeldung -> What is Abstract Data Type?](https://www.baeldung.com/cs/adt)

- [Stack Overflow -> Running Pytest inside a Jupyter notebook](https://stackoverflow.com/questions/41304311/running-pytest-test-functions-inside-a-jupyter-notebook)

- [GitHub -> IPytest](https://github.com/chmp/ipytest)

- [Pytest -> Examples -> Parametrising tests](https://docs.pytest.org/en/7.1.x/example/parametrize.html)

### Solutions

1. The implementation of natural numbers abstract data type -> writing a `NaturalNumber` data structure.

In [14]:
class NaturalNumber:
    def __init__(self, value):
        validate_positive_integer(value)
        self._val = value

    @property
    def value(self) -> int:
        """self.value means self._val"""
        return self._val

    def _get_value(self, other):
        """a helper for operations"""
        if isinstance(other, NaturalNumber):
            return other.value
        return other

    def __add__(self, other):
        """self + other"""
        return NaturalNumber(self._val + self._get_value(other))

    def __iadd__(self, other):
        """self += other"""
        return self + other

    def __radd__(self, other):
        """other + self"""
        return self + other  # calling __add__

    def __eq__(self, other):
        return self._val == self._get_value(other)

    def __mul__(self, other):
        """self * other"""
        return NaturalNumber(self._val * self._get_value(other))

    def __imul__(self, other):
        """self *= other"""
        return self * other

    def __rmul__(self, other):
        """other * self"""
        return self * other

    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}({self._val})"

    def __str__(self) -> str:
        return str(self._val)

    def __sub__(self, other):
        """self - other"""
        return NaturalNumber(self.value - self._get_value(other))

    def __isub__(self, other):
        """self -= other"""
        return self - other

    def __rsub__(self, other):
        """other - self"""
        return self - other
    
    def __truediv__(self, other):
        """self / other"""
        raise NotImplementedError

    def __itruediv__(self, other):
        """self / other"""
        raise NotImplementedError

    def __rtruediv__(self, other):
        """other / self"""
        raise NotImplementedError
    
    def __floordiv__(self, other):
        """self // other"""
        res: float = self._val / self._get_value(other)
        if not res.is_integer():
            raise ValueError(f"{res} is out of natural numbers")
        return NaturalNumber(int(res))

    def __ifloordiv__(self, other):
        """self // other"""
        return self // other

    def __rfloordiv__(self, other):
        """other // self"""
        return self // other

In [15]:
%%ipytest -s


class TestNaturalNumbers:
    """Test Suite for testing Natural Numbers."""

    @pytest.mark.parametrize(
        ("nbr", "expectation"),
        [
            (-1, pytest.raises(ValueError)),
            (0, pytest.raises(ValueError)),
            (1.0, pytest.raises(TypeError)),
        ]
    )
    def test_init(self, nbr, expectation):
        with expectation:
            NaturalNumber(nbr)

    @pytest.mark.parametrize(
        ("nbr1", "nbr2", "answer"),
        [
            (1, 1, 2),
        ]
    )
    def test_add(self, nbr1, nbr2, answer):
        n1, n2 = NaturalNumber(nbr1), NaturalNumber(nbr2)
        assert n1 + n2 == n2 + n1 == answer
        n1 += n2
        assert n1 == answer

    @pytest.mark.parametrize(
        ("nbr1", "nbr2", "answer"),
        [
            (1, 1, 1),
            (2, 1, 2),
            (2, 2, 4),
        ]
    )
    def test_mul(self, nbr1, nbr2, answer):
        n1, n2 = NaturalNumber(nbr1), NaturalNumber(nbr2)
        assert n1 * n2 == n2 * n1 == answer
        n1 *= n2
        assert n1 == answer

    @pytest.mark.parametrize(
        ("nbr1", "nbr2", "answer", "expectation"),
        [
            (2, 1, 1, does_not_raise()),
            # this row shows a possible design failure, but anyway
            (2, 2, 0, pytest.raises(ValueError)),
            (2, 3, -1, pytest.raises(ValueError))
        ]
    )
    def test_sub(self, nbr1, nbr2, answer, expectation):
        with expectation:
            n1, n2 = NaturalNumber(nbr1), NaturalNumber(nbr2)
            assert n1 - n2 == answer
            n1 -= n2
            assert n1 == answer

    @pytest.mark.parametrize(
        ("nbr1", "nbr2", "answer", "expectation"),
        [
            (1, 1, 1, does_not_raise()),
            (2, 2, 1, does_not_raise()),
            (5, 3, 5//3, pytest.raises(ValueError)),
            (3, 5, 3//5, pytest.raises(ValueError)),
        ]
    )
    def test_div(self, nbr1, nbr2, answer, expectation):
        with expectation:
            n1, n2 = NaturalNumber(nbr1), NaturalNumber(nbr2)
            assert n1 // n2 == answer
            n1 //= n2
            assert n1 == answer

..............
14 passed in 0.02s


2. You can do it yourself

3. Will be mostly covered later when uncovering ways to interchange recursive and iterative implementations.

In [16]:
def _get_nth_triangular_number_recursive(nth: int) -> int:
    if not nth:
        return 1
    return _get_nth_triangular_number_recursive(nth - 1) + nth


# just redefining the function
def get_triangular_number_recursive(nth: int) -> int:
    """Returns the nth triangular number.

    The indexation starts from zero, so T(0) = 1, T(1) = 3 and so forth.
    """

    validate_non_negative_integer(nth)
    return _get_nth_triangular_number_recursive(nth)


def get_triangular_number_iterative(nth: int) -> int:
    """Returns the nth triangular number.

    The indexation starts from zero, so T(0) = 1, T(1) = 3 and so forth.
    """

    validate_non_negative_integer(nth)
    sum_ = 1
    for _ in range(1, nth):
        sum_ += (sum_ + 1)
    return sum_

In [17]:
%%ipytest

# https://github.com/chmp/ipytest
@pytest.mark.parametrize(
    ("number", "answer", "expectation"),
    [
        (-21, None, pytest.raises(ValueError)),
        (4.2, None, pytest.raises(TypeError)),
        (0, 1, does_not_raise()),
        (1, 3, does_not_raise()),
        (2, 6, does_not_raise()),
        (3, 10, does_not_raise()),
    ]
)
def test_get_nth_triangular_number(
    number, answer, expectation
):
    with expectation:
        res_rec = get_triangular_number_recursive(number)
        res_iter = get_triangular_number_iterative(number)
        res_rec == res_iter == answer

......                                                                                       [100%]
6 passed in 0.01s


That is all for now.